In [1]:
import keras
import collections
import numpy as np
import os
import random
import re
import string
import time

Using Theano backend.
Using gpu device 0: GeForce GTX 950 (CNMeM is enabled with initial size: 66.0% of memory, cuDNN 5105)
/usr/local/lib/python2.7/dist-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


DATA PREPARATION

In [2]:
def remove_hidden(l):
    return filter(lambda s: not s[0] == '.', l)

d = "//home//xgamer//OANC-GrAF//data//written_1//journal//slate"

def get_files(d):
    files = os.listdir(d)
    files = remove_hidden(files)
    return map(lambda f: d + "//" + f, files)

dirs = get_files(d)
files = map(get_files, dirs)
files = [f for l in files for f in l]
files = filter(lambda f: f[-3:] == 'txt', files)

print len(files)

texts = []
for f in files:
    f = open(f)
    texts.append('\n'.join(f.readlines()[7:]))
    f.close()

print sum([len(text) for text in texts])

4531
27629896


In [3]:
def clean_text(text):
    text = text.strip()
    text = text.replace('\n', ' ')
    text = text.replace('--', ' - ')
    text = text.replace('\t', ' ')
    text = re.sub(r'\ \ +', r' ', text)
    text = filter(lambda char: char in string.printable, text)
    return text

texts = map(clean_text, texts)
print sum([len(text) for text in texts])

25425786


In [4]:
print texts[545]

Trustbusters Have Job Security The Dec. 14 edition of "Today's Papers" notes a Washington Post report that David Boies, the lead Justice Department attorney in the Microsoft case, has lowered the hourly fee that he is charging the government. The column asks whether this is legal, if Boies' motive was to keep his share of the work. The answer can be found in the definitive rhyming opus on antitrust law, R.W. Grant's Tom Smith and His Incredible Bread Machine : You're gouging on your prices if You charge more than the rest. But it's unfair competition If you think you can charge less! A second point that we would make To help avoid confusion Don't try to charge the same amount! That would be collusion! In short, Boies' reduction of his fee constitutes unfair competition. Of course, if he'd left his fee unchanged, that would be collusion, though in this case he'd be colluding with himself and would eventually go blind. How's that for legal clarity? - Sam Kazman Washington Bork and Mindy 

In [5]:
alphabet = collections.defaultdict(bool)

for text in texts:
    for char in text:
        alphabet[char] = True

alphabet = [char for char in alphabet if alphabet[char] == True]
alphabet.sort()

# Add a start of sequence token \t
alphabet.append('\t')
# Add an end of sequence token \n
alphabet.append('\n')

texts = map(lambda text: '\t' + text + '\n', texts)
print alphabet

char_to_index = {}
index_to_char = {}
for i in xrange(len(alphabet)):
    index_to_char[i] = alphabet[i]
    char_to_index[alphabet[i]] = i
print index_to_char
print char_to_index

[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '}', '~', '\t', '\n']
{0: ' ', 1: '!', 2: '"', 3: '#', 4: '$', 5: '%', 6: '&', 7: "'", 8: '(', 9: ')', 10: '*', 11: '+', 12: ',', 13: '-', 14: '.', 15: '/', 16: '0', 17: '1', 18: '2', 19: '3', 20: '4', 21: '5', 22: '6', 23: '7', 24: '8', 25: '9', 26: ':', 27: ';', 28: '<', 29: '=', 30: '>', 31: '?', 32: '@', 33: 'A', 34: 'B', 35: 'C', 36: 'D', 37: 'E', 38: 'F', 39: 'G', 40: 'H', 41: 'I', 42: 'J', 43: 'K', 44: 'L', 45: 'M', 46: 'N', 47: 'O', 48: 'P', 49: 'Q', 50: 'R', 51: 'S', 52: 'T', 53: 'U', 54: 'V', 55: 'W', 56: 'X', 57: 'Y', 58:

In [6]:
text_seq = ''.join(texts)
sample_len = 200

def sample(text, length=sample_len + 1):
    i = random.randint(0, len(text) - length - 1)
    return text[i:i+length]

def toseqs(string, alphabet=alphabet):
    assert len(string) > 1
    n = len(string) - 1
    data = np.zeros(n, dtype='int')
    label = np.zeros(len(alphabet), dtype='uint8')
    for i in xrange(n):
        cur = string[i]
        data[i] = char_to_index[cur]
    nxt = string[i+1]
    label[char_to_index[nxt]] = 1
    return data, label

s = sample(text_seq)
print s
x,y = toseqs(s)
print x
print y

def tostring(seq, alphabet=alphabet):
    string = []
    for i in xrange(seq.shape[0]):
        string.append(index_to_char[np.argmax(seq[i])])
    return ''.join(string)

print len(x), len(tostring(x))

 quotidian proximity that the role affords, has demonstrated a proven knack for understated aptness in its selections. Ronald "Rawhide" Reagan. Dan "Scorecard" Quayle. Edward "Sunburn" Kennedy. Jimmy "
[ 0 81 85 79 84 73 68 73 65 78  0 80 82 79 88 73 77 73 84 89  0 84 72 65 84
  0 84 72 69  0 82 79 76 69  0 65 70 70 79 82 68 83 12  0 72 65 83  0 68 69
 77 79 78 83 84 82 65 84 69 68  0 65  0 80 82 79 86 69 78  0 75 78 65 67 75
  0 70 79 82  0 85 78 68 69 82 83 84 65 84 69 68  0 65 80 84 78 69 83 83  0
 73 78  0 73 84 83  0 83 69 76 69 67 84 73 79 78 83 14  0 50 79 78 65 76 68
  0  2 50 65 87 72 73 68 69  2  0 50 69 65 71 65 78 14  0 36 65 78  0  2 51
 67 79 82 69 67 65 82 68  2  0 49 85 65 89 76 69 14  0 37 68 87 65 82 68  0
  2 51 85 78 66 85 82 78  2  0 43 69 78 78 69 68 89 14  0 42 73 77 77 89  0]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

NN MODEL

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers.embeddings import Embedding
from keras.optimizers import RMSprop, SGD
import bisect

In [8]:
embedding_size = len(alphabet)
n_lstm_cells = 1000
dropout_perc = 0.1

model = Sequential()
print sample_len, len(alphabet)
model.add(Embedding(len(alphabet), embedding_size, input_length=sample_len))
model.add(LSTM(n_lstm_cells))
model.add(Dropout(dropout_perc))
model.add(Dense(len(alphabet)))
model.add(Activation('softmax'))
#opt = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
opt = RMSprop()
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

200 96


In [9]:
batch_size = 64
n_batches_per_epoch = 5001

def get_batch(size, text=text_seq):
    xs = np.zeros((size, sample_len), dtype='int')
    ys = np.zeros((size, len(alphabet)), dtype='uint8')
    for i in xrange(size):
        s = sample(text)
        xs[i],ys[i] = toseqs(s)
    return xs, ys

valid_x, valid_y = get_batch(batch_size)
seen_samples = 0
start = time.time()

In [14]:
mu_acc = 0.0
e_start = time.time()
start_samples = seen_samples
output_every_n = 100

for i in xrange(n_batches_per_epoch):
    x,y = get_batch(batch_size)
    seen_samples += batch_size
    _,acc =  model.train_on_batch(x, y)
    mu_acc += acc
    if i % output_every_n == 0 and i > 0:
        print "Saw " + str(seen_samples) + " samples."
        t = time.time()
        print "Ran " + str(t-start) + " seconds. (This epoch: " +\
            str((seen_samples-start_samples)*1./(t-e_start)) + " samples / second)"
        print "Train accuracy: " + str(1./float(output_every_n)*mu_acc)
        mu_acc = 0.0
        _,acc = model.test_on_batch(valid_x, valid_y)
        print "Test accuracy: " + str(acc)
        print

model.save("charlevel_b2.h5")

Saw 326528 samples.
Ran 3673.57299304 seconds. (This epoch: 94.2069131202 samples / second)
Train accuracy: 0.490625
Test accuracy: 0.59375

Saw 332928 samples.
Ran 3742.15835118 seconds. (This epoch: 93.7607414666 samples / second)
Train accuracy: 0.50125
Test accuracy: 0.53125

Saw 339328 samples.
Ran 3810.96644211 seconds. (This epoch: 93.5107631586 samples / second)
Train accuracy: 0.4871875
Test accuracy: 0.5625

Saw 345728 samples.
Ran 3879.58535719 seconds. (This epoch: 93.4502916863 samples / second)
Train accuracy: 0.49578125
Test accuracy: 0.53125

Saw 352128 samples.
Ran 3947.83170414 seconds. (This epoch: 93.5155022151 samples / second)
Train accuracy: 0.48125
Test accuracy: 0.546875

Saw 358528 samples.
Ran 4016.30265307 seconds. (This epoch: 93.50797768 samples / second)
Train accuracy: 0.4875
Test accuracy: 0.578125

Saw 364928 samples.
Ran 4084.93296719 seconds. (This epoch: 93.4715551915 samples / second)
Train accuracy: 0.48984375
Test accuracy: 0.59375

Saw 371328 sa

In [28]:
text_len = 500
n_samples = 5
seed_string = "\tBill and Hillary Clinton "

def tochar_prob(output):
    summed = []
    _sum = 0.0
    output = output[0]
    output = output**2
    output /= sum(output)
    for i in xrange(output.shape[0]):
        _sum += output[i]
        summed.append(_sum)
    choice = random.random()
    i = bisect.bisect(summed, choice)
    return index_to_char[i]
        
def generate(length, seed='\t', model=model):
    seq = np.zeros((1, sample_len))
    padding_len = sample_len - len(seed)
    if padding_len < 1:
        padding_len = 0
    for i in xrange(padding_len):
        seq[0,i] = 0
    for i in xrange(len(seed)):
        seq[0,padding_len+i] = char_to_index[seed[i]]
        
    string = list(seed_string)
    for i in xrange(length):
        c = tochar_prob(model.predict(seq))
        string.append(c)
        if c == '\n':
            break
        seq = np.roll(seq, -1, axis=1)
        seq[0,-1] = char_to_index[c]
    return ''.join(string)

for i in xrange(n_samples):
    print generate(text_len, seed_string)

	Bill and Hillary Clinton could dragn the president that the president is a lottered and reports and someone of the and of the prission of the court to get it was a companies and behind the but one of the some way pronection in the anticle in China that we said the start to take the country, his last most beungress of a smarter and in convention of the committed mention of a resuarches to the last term of the inventional design in the can be a companious morning to the simple of the New York Times Backs with the states t
	Bill and Hillary Clinton said. The country. The secret and self-corperction to friend that the congressional prompity of the last describes are noted a fair of the policy called the resson that the years ago, which centrally religing the books not be a time which are disingents to the president of the way the boster to be an as an a high-states of problems a beautic religious as which friends and every broke as a sping of the discussion to be a to allowed the work for